#Vosk

In [ ]:
!pip install vosk


In [ ]:
!pip install pyaudio

  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyaudio
Failed to build pyaudio
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pyaudio)


In [ ]:
from vosk import Model, KaldiRecognizer
import pyaudio
import json

model = Model("model-id")  # Unduh model Bahasa Indonesia dari Vosk
rec = KaldiRecognizer(model, 16000)
mic = pyaudio.PyAudio().open(format=pyaudio.paInt16, channels=1, rate=16000, input=True, frames_per_buffer=8192)

while True:
    data = mic.read(4096)
    if rec.AcceptWaveform(data):
        result = json.loads(rec.Result())
        text = result.get("text", "").lower()
        print("Teks:", text)
        if any(kw in text for kw in ["tolong", "jangan", "help", "sakit"]):
            print("🚨 SOS Detected! Kirim bantuan sekarang!")
            # Tambah aksi pengiriman lokasi atau alert


#Whisper (Open AI)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install openai-whisper

In [ ]:
import whisper

# Load the Whisper model
# This might take some time on the first run as it downloads the model
model = whisper.load_model("base")

# Transcribe the audio file
# Make sure you have an audio file named "audio.mp3" in your environment
result = model.transcribe("/content/common_voice_id_19051304.mp3")

# Print the transcribed text
print(result["text"])

 apakah ini sangat sakit?


In [ ]:
# Daftar kata kunci darurat yang ingin kamu deteksi
emergency_keywords = ["tolong", "bantu saya", "saya diserang", "help", "emergency", "darurat"]

# Ubah hasil transkripsi ke huruf kecil untuk pencocokan
transcribed_text = result["text"].lower()

# Cek apakah salah satu keyword ada di dalam teks
if any(keyword in transcribed_text for keyword in emergency_keywords):
    print("🚨 Deteksi situasi darurat! Kirim pesan SOS!")
    # Di sini kamu bisa tambah aksi, misal kirim WA/SMS/notifikasi
else:
    print("✅ Tidak ada kata darurat terdeteksi.")


✅ Tidak ada kata darurat terdeteksi.


In [ ]:
!pip install pandas scikit-learn joblib

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import joblib

In [ ]:
#  Load CSV dataset
df = pd.read_csv("/content/drive/MyDrive/Datathon/Tim/Dokumen/Dataset/darurat_aman_dataset.csv")  # Pastikan file ini ada

# Split data
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# TF-IDF vektor
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Model klasifikasi
classifier = LogisticRegression()
classifier.fit(X_train_vec, y_train)

# Evaluasi model
y_pred = classifier.predict(X_test_vec)
print("Akurasi:", accuracy_score(y_test, y_pred))
print("Laporan Klasifikasi:\n", classification_report(y_test, y_pred))

# Simpan model dan vectorizer
joblib.dump(classifier, "bahaya_classifier_model.joblib")
joblib.dump(vectorizer, "tfidf_vectorizer.joblib")


Akurasi: 0.926829268292683
Laporan Klasifikasi:
               precision    recall  f1-score   support

        aman       0.86      1.00      0.93        19
     darurat       1.00      0.86      0.93        22

    accuracy                           0.93        41
   macro avg       0.93      0.93      0.93        41
weighted avg       0.94      0.93      0.93        41



['tfidf_vectorizer.joblib']

In [ ]:
# Load model
clf = joblib.load("bahaya_classifier_model.joblib")
vec = joblib.load("tfidf_vectorizer.joblib")

# Prediksi contoh kalimat
kalimat = ["Tolong"]
kalimat_vec = vec.transform(kalimat)
hasil = clf.predict(kalimat_vec)
print("Prediksi:", hasil[0])


Prediksi: darurat


#Fine tunning

In [ ]:
! pip install git+https://github.com/openai/whisper.git


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-fzj36poq
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-fzj36poq
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import io
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import urllib
import tarfile
import whisper
import torchaudio

from scipy.io import wavfile
from tqdm.notebook import tqdm


pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 1000
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import ipywidgets as widgets

languages = {"af_za": "Afrikaans", "am_et": "Amharic", "ar_eg": "Arabic", "as_in": "Assamese", "az_az": "Azerbaijani", "be_by": "Belarusian", "bg_bg": "Bulgarian", "bn_in": "Bengali", "bs_ba": "Bosnian", "ca_es": "Catalan", "cmn_hans_cn": "Chinese", "cs_cz": "Czech", "cy_gb": "Welsh", "da_dk": "Danish", "de_de": "German", "el_gr": "Greek", "en_us": "English", "es_419": "Spanish", "et_ee": "Estonian", "fa_ir": "Persian", "fi_fi": "Finnish", "fil_ph": "Tagalog", "fr_fr": "French", "gl_es": "Galician", "gu_in": "Gujarati", "ha_ng": "Hausa", "he_il": "Hebrew", "hi_in": "Hindi", "hr_hr": "Croatian", "hu_hu": "Hungarian", "hy_am": "Armenian", "id_id": "Indonesian", "is_is": "Icelandic", "it_it": "Italian", "ja_jp": "Japanese", "jv_id": "Javanese", "ka_ge": "Georgian", "kk_kz": "Kazakh", "km_kh": "Khmer", "kn_in": "Kannada", "ko_kr": "Korean", "lb_lu": "Luxembourgish", "ln_cd": "Lingala", "lo_la": "Lao", "lt_lt": "Lithuanian", "lv_lv": "Latvian", "mi_nz": "Maori", "mk_mk": "Macedonian", "ml_in": "Malayalam", "mn_mn": "Mongolian", "mr_in": "Marathi", "ms_my": "Malay", "mt_mt": "Maltese", "my_mm": "Myanmar", "nb_no": "Norwegian", "ne_np": "Nepali", "nl_nl": "Dutch", "oc_fr": "Occitan", "pa_in": "Punjabi", "pl_pl": "Polish", "ps_af": "Pashto", "pt_br": "Portuguese", "ro_ro": "Romanian", "ru_ru": "Russian", "sd_in": "Sindhi", "sk_sk": "Slovak", "sl_si": "Slovenian", "sn_zw": "Shona", "so_so": "Somali", "sr_rs": "Serbian", "sv_se": "Swedish", "sw_ke": "Swahili", "ta_in": "Tamil", "te_in": "Telugu", "tg_tj": "Tajik", "th_th": "Thai", "tr_tr": "Turkish", "uk_ua": "Ukrainian", "ur_pk": "Urdu", "uz_uz": "Uzbek", "vi_vn": "Vietnamese", "yo_ng": "Yoruba"}
selection = widgets.Dropdown(
    options=[("Select language", None), ("----------", None)] + sorted([(f"{v} ({k})", k) for k, v in languages.items()]),
    value="id_id",
    description='Language:',
    disabled=False,
)

selection

Dropdown(description='Language:', index=32, options=(('Select language', None), ('----------', None), ('Afrika…

In [ ]:
lang = selection.value
language = languages[lang]

assert lang is not None, "Please select a language"
print(f"Selected language: {language} ({lang})")

Selected language: Indonesian (id_id)


In [ ]:
def download(url: str, target_path: str):
    with urllib.request.urlopen(url) as source, open(target_path, "wb") as output:
        with tqdm(total=int(source.info().get("Content-Length")), ncols=80, unit='iB', unit_scale=True, unit_divisor=1024) as loop:
            while True:
                buffer = source.read(8192)
                if not buffer:
                    break

                output.write(buffer)
                loop.update(len(buffer))


class Fleurs(torch.utils.data.Dataset):
    """
    A simple class to wrap Fleurs and subsample a portion of the dataset as needed.
    """
    def __init__(self, lang, split="test", subsample_rate=1, device=DEVICE):
        url = f"https://storage.googleapis.com/xtreme_translations/FLEURS102/{lang}.tar.gz"
        tar_path = os.path.expanduser(f"~/.cache/fleurs/{lang}.tgz")
        os.makedirs(os.path.dirname(tar_path), exist_ok=True)

        if not os.path.exists(tar_path):
            download(url, tar_path)

        all_audio = {}
        with tarfile.open(tar_path, "r:gz") as tar:
            for member in tar.getmembers():
                name = member.name
                if name.endswith(f"{split}.tsv"):
                    labels = pd.read_table(tar.extractfile(member), names=("id", "file_name", "raw_transcription", "transcription", "_", "num_samples", "gender"))

                if f"/{split}/" in name and name.endswith(".wav"):
                    audio_bytes = tar.extractfile(member).read()
                    all_audio[os.path.basename(name)] = wavfile.read(io.BytesIO(audio_bytes))[1]

        self.labels = labels.to_dict("records")[::subsample_rate]
        self.all_audio = all_audio
        self.device = device

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, item):
        record = self.labels[item]
        audio = torch.from_numpy(self.all_audio[record["file_name"]].copy())
        text = record["transcription"]

        return (audio, text)

In [ ]:
dataset = Fleurs(lang, subsample_rate=10)  # subsample 10% of the dataset for a quick demo


  0%|                                              | 0.00/2.21G [00:00<?, ?iB/s]

#Running inference on the dataset using a medium Whisper model

The following will take a few minutes to transcribe and translate utterances in the dataset.



In [ ]:
model = whisper.load_model("medium")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

100%|█████████████████████████████████████| 1.42G/1.42G [00:26<00:00, 58.2MiB/s]


Model is multilingual and has 762,321,920 parameters.


In [ ]:

options = dict(language=language, beam_size=5, best_of=5)
transcribe_options = dict(task="transcribe", **options)
translate_options = dict(task="translate", **options)

In [ ]:

references = []
transcriptions = []
translations = []

for audio, text in tqdm(dataset):
    transcription = model.transcribe(audio, **transcribe_options)["text"]
    translation = model.transcribe(audio, **translate_options)["text"]

    transcriptions.append(transcription)
    translations.append(translation)
    references.append(text)

  0%|          | 0/69 [00:00<?, ?it/s]

In [ ]:

data = pd.DataFrame(dict(reference=references, transcription=transcriptions, translation=translations))
data

,reference,transcription,translation
0,strategi tersebut terbukti efektif memutus pasokan penting militer dan sipil meskipun blokade ini melanggar hukum internasional yang diterima secara umum yang dikodifikasi oleh beberapa perjanjian internasional selama dua abad terakhir,"Strategi tersebut terbukti efektif, memutus pasokan penting militer dan sipil, meskipun blokade ini melanggar hukum internasional yang diterima secara umum yang dikodifikasi oleh beberapa perjanjian internasional selama dua abad terakhir.","The strategy is proven effective, cutting off the importance of military and civilian troops. Although this blockade violates international law, which is generally accepted, which has been codified by several international agreements for the last two centuries."
1,namun ada banyak hal tentang burung yang masih terlihat seperti dinosaurus,"Namun, ada banyak hal tentang burung yang masih terlihat seperti dinosaurus.","However, there are many things about birds that still look like dinosaurs."
2,perdebatan dipicu kontroversi terkait pengeluaran dana bantuan dan rekonstruksi pasca-badai katrina yang dilabeli oleh kelompok konservatif fiskal sebagai kesepakatan new orleans bush,Perdebatan dipicu kontroversi terkait mengeluaran dana bantuan dan rekonstruksi pasca badai Katrina yang dilabeli oleh kelompok konservatif fiskal sebagai kesepakatan New Orleans Bush.,The debate was controversial regarding the release of help fund and reconstruction of PASCA Badai Katrina which was labeled by the conservative fiscal group as New Orleans Bush's partnership.
3,pengumuman itu dibuat setelah trump melakukan percakapan telepon dengan presiden turki recep tayyip erdoäÿan,Pengumuman itu dibuat setelah Trump melakukan percakapan telepon dengan Presiden Turki Recep Tayyip Erdogan.,The announcement was made after Trump made a phone conversation with Turkish President Recep Tayyip Erdogan.
4,area yang luas di sebelah utara sangat jarang ditempati dan beberapa bahkan adalah hutan belantara yang hampir tak dapat ditempati,area yang luas di sebelah utara sangat jarang ditempati dan beberapa bahkan adalah hutan pelantara yang hampir tak dapat ditempati,"The large area on the north side is very rare to be located, and some even are desert forests that are almost impossible to be located."
5,jadi notasi itu kemungkinan besar disisipkan cuma sebagai label,"Jadi, notasi itu kemungkinan besar disisitkan cuma sebagai level.","So, the notation is likely to be used only as a level."
6,namun karena ditempatkan di dataran tinggi tropis hanya beberapa derajat di utara khatulistiwa anda perlu berurusan dengan panas selalu maupun matahari yang kuat ketika langit cerah lebih jarang.\t n a m u n | k a r e n a | d i t e m p a t k a n | d i | d a t a r a n | t i n g g i | t r o p i s | h a n y a | b e b e r a p a | d e r a j a t | d i | u t a r a | k h a t u l i s t i w a | a n d a | p e r l u | b e r u r u s a n | d e n g a n | p a n a s | s e l a l u | m a u p u n | m a t a h a r i | y a n g | k u a t | k e t i k a | l a n g i t | c e r a h | l e b i h | j a r a n g . |,"Namun, karena ditempatkan di dataran tinggi tropis hanya beberapa derajat di utara katulistiwa, Anda perlu berurusan dengan panas selalu maupun matahari yang kuat ketika langit cerah lebih jarang.","However, because it is located on a tropical highland, only a few degrees in the north of the Tewa River, you need to take care of it hot, always, even the strong sun when the sky is bright, is more rare."
7,satelit itu dikirim ke luar angkasa oleh roket ilmuwan menggunakan teleskop di luar angkasa karena atmosfer bumi mengubah sebagian cahaya dan pandangan kita,Satelit itu dikirim ke luar angkasa oleh roket ilmuwan menggunakan teleskop di luar angkasa karena atas mosfair bumi mengubah sebagian cahaya dan pandangan kita.,The satellite was sent to outer space by rocket Ilmuan using a telescope in outer space because Earth's atmosphere changes some of our light and view.
8,jangan mencoreng situs dengan 